In [ ]:
# Now you should have the monacle result output table
# Now just visualize it in python
terms = pd.read_csv('monacle_result') # I saved that table in csv
terms['-logQ'] = -np.log10(terms['q_value'])
terms = terms[terms['-logQ']>0]
fc_thresh = 1
lowqval_de = terms[np.array(terms['-logQ']>=-np.log10(0.05)) & np.array(abs(terms['normalized_effect'])>=fc_thresh)]

In [ ]:
##### Volcano plot of results
from adjustText import adjust_text
plt.rcParams['figure.figsize'] = [8,8]
plt.rcParams.update({'font.size': 14, 'font.weight': 'bold','axes.linewidth':2})
lowqval_de = terms[np.array(terms['-logQ']>=-np.log10(0.05)) & np.array(abs(terms['normalized_effect'])>=fc_thresh)]
other_de = terms[np.array(terms['-logQ']<-np.log10(0.05)) | np.array(abs(terms['normalized_effect'])<fc_thresh)] #actually that's other genes (non sig de)

fig, ax = plt.subplots()
sb.regplot(other_de['normalized_effect'], other_de['-logQ'], fit_reg=False, scatter_kws={'s':50, 'edgecolors': 'black', 'linewidths': 0.5})
sb.regplot(lowqval_de['normalized_effect'], lowqval_de['-logQ'], fit_reg=False, scatter_kws={'s':50,'edgecolors': 'black', 'linewidths': 0.5})
ax.set_xlabel("Normalized log2 FC", fontsize=30)
ax.set_ylabel("-log Q-value", fontsize=30)
ax.tick_params(labelsize=15)
# below is just for one subset, but the same code apply to different comparison
# With some adjustment on the label density if necessary
ax.set_title('IL7R+ TRM Carcinoma vs Normal',size = 25)
    
    
    
# Label names and positions
x = [i for i in lowqval_de['normalized_effect']]
y = [i*1.02 for i in lowqval_de['-logQ']]
labels = lowqval_de['gene_short_name']
# Show only some labels to avoid overcrowding the figure
to_keep = np.where((np.array(lowqval_de['gene_short_name'].str.startswith('RP'))
                    |np.array(lowqval_de['gene_short_name'].str.startswith('MT'))
                     |(np.array(lowqval_de['normalized_effect']<fc_thresh*1.5)
                       &np.array(lowqval_de['normalized_effect']>0)
                           &np.array(lowqval_de['-logQ']<50))
                    |(np.array(abs(lowqval_de['normalized_effect'])<fc_thresh*1.5)
                      &np.array(lowqval_de['normalized_effect']<0)
                           &np.array(lowqval_de['-logQ']<50))
                   |np.array(lowqval_de['-logQ']<5) |np.array(lowqval_de['-logQ']>70))==0)

labels = labels.iloc[to_keep]
texts = []

#Highlight these genes because they are associated with cytotoxity/exhaustion/etc
for i,txt in enumerate(labels):
    if txt not in ['GZMB','GNLY','NKG7','GZMH','CD74','VCAM1','KLF2','CTLA4','ITGB2','IFNG',
               'ZNF683','FCER1G','GZMK','KLRG1','FGFBP2','FCGR3A','IL7R','PDCD1','CCL4','KLRK1',
              'CD81','CD82','CD83','NFKBIA']:
        texts.append(ax.text(x[to_keep[0][i]], y[to_keep[0][i]], txt,size = 11, color= '#904800', weight = 'bold'))
    
for i,txt in enumerate(labels):    
    if txt in ['GZMB','GNLY','NKG7','GZMH','CD74','VCAM1','KLF2','CTLA4','ITGB2','IFNG',
               'ZNF683','FCER1G','GZMK','KLRG1','FGFBP2','FCGR3A','IL7R','PDCD1','CCL4','KLRK1',
              'CD81','CD82','CD83','NFKBIA','HAVCR2']:
        texts.append(ax.text(x[to_keep[0][i]], y[to_keep[0][i]], txt,size = 15, color= '#0E46E2', weight = 'bold', bbox=dict(boxstyle="Square,pad=0.1", fc="white", ec="b", lw=2, alpha = 0.8)))
        
plt.ylim([-0.01,90])
plt.xlim([-3,7.8])
adjust_text(texts,arrowprops=dict(arrowstyle='-', color='red'))
#ax.xaxis.label.set_fontweight('bold')
#ax.yaxis.label.set_fontweight('bold')
#ax.yaxis.label.set_fontsize(22)
#ax.annotate(txt, (x[i], y[i]),size = 10,bbox=dict(boxstyle="Square,pad=0.3", fc="orange", ec="b", lw=2, alpha = 0.6))


plt.axhline(y=-np.log10(0.05), color="orange", linestyle="--")
plt.axvline(x = -fc_thresh, color="orange", linestyle="--")
plt.axvline(x = fc_thresh, color="orange", linestyle="--")
plt.savefig('DE.png')
#plt.show()